У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [ ]:
#import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTENC

from imblearn.combine import SMOTETomek

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

In [ ]:
# Load dataset

data_url = 'https://raw.githubusercontent.com/roldugina/Machine_learning_course/main/customer_segmentation_train.csv'
raw_df = pd.read_csv(data_url)
raw_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [ ]:
# Check if there are null values

raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [ ]:
# Calculating the percentage of null values in column

np.round(raw_df.isnull().sum()/raw_df.shape[0]*100, 2)

ID                  0.00
Gender              0.00
Ever_Married        1.74
Age                 0.00
Graduated           0.97
Profession          1.54
Work_Experience    10.28
Spending_Score      0.00
Family_Size         4.15
Var_1               0.94
Segmentation        0.00
dtype: float64

Відсоток пропущених значень у більшості випадків не перевищує 10%. І у одному випадку незначно перевищує 10%, тому можна застосовувати імпутацію.

In [ ]:
# Convert 'Yes / No' categorical types into numeric

raw_df['Ever_Married'] = raw_df['Ever_Married'].map({'Yes':1,'No':0})
raw_df['Graduated'] = raw_df['Graduated'].map({'Yes':1,'No':0})
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   float64
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   float64
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(4), int64(2), object(5)
memory usage: 693.5+ KB


In [ ]:
# Split the dataset into training and testing sets

train_df, test_df = train_test_split(raw_df, test_size = 0.2, random_state = 42, stratify = raw_df.Segmentation )
print('Мітки класів в тренувальній вибірці\n',train_df.Segmentation.value_counts(normalize = True),'\n')
print('Мітки класів в валідаційній вибірці\n',test_df.Segmentation.value_counts(normalize = True), '\n')

Мітки класів в тренувальній вибірці
 Segmentation
D    0.281066
A    0.244500
C    0.244190
B    0.230245
Name: proportion, dtype: float64 

Мітки класів в валідаційній вибірці
 Segmentation
D    0.281289
A    0.244114
C    0.244114
B    0.230483
Name: proportion, dtype: float64 



In [ ]:
# Create input and target data

def create_x_y(inputs, targets, numeric_cols, encoded_cols, target_col):

  X = inputs[numeric_cols+encoded_cols]
  y = targets[target_col]

  return X, y

In [ ]:
# Create input and target column lists

input_cols = train_df.columns.tolist()[1:-1]
target_col = train_df.columns.tolist()[-1]

train_inputs = train_df[input_cols]
test_inputs = test_df[input_cols]

In [ ]:
# Identify numeric and categorical columns

numeric_cols = train_df.select_dtypes(include = np.number).columns.tolist()[1:]
categorical_cols = train_df.select_dtypes(include = 'object').columns.tolist()[:-1]

# Impute numeric data with simpleimputer

imputer_num = SimpleImputer(strategy = 'mean')
imputer_num.fit(train_df[numeric_cols])
train_inputs.loc[:, numeric_cols] = imputer_num.transform(train_df[numeric_cols])
test_inputs.loc[:, numeric_cols] = imputer_num.transform(test_df[numeric_cols])

# Impute categorical data with simpleimputer

imputer_cat = SimpleImputer(strategy = 'most_frequent')
imputer_cat.fit(train_df[categorical_cols])
train_inputs.loc[:, categorical_cols] = imputer_cat.transform(train_df[categorical_cols])
test_inputs.loc[:, categorical_cols] = imputer_cat.transform(test_df[categorical_cols])

# Transform categorical data with onehotencoder

onehot = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')
onehot.fit(train_inputs[categorical_cols])
encoded_cols = onehot.get_feature_names_out(categorical_cols).tolist()
train_inputs.loc[:, encoded_cols] = onehot.transform(train_df[categorical_cols])
test_inputs.loc[:, encoded_cols] = onehot.transform(test_df[categorical_cols])

# Create final train and test sets (after preprocessing)

X_train, y_train = create_x_y(train_inputs, train_df, numeric_cols, encoded_cols, target_col)
X_test, y_test = create_x_y(test_inputs, test_df, numeric_cols, encoded_cols, target_col)

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [ ]:
# Perform SMOTENC oversampling

categorical_indices = [X_train.columns.get_loc(col) for col in encoded_cols]
smotenc = SMOTENC(categorical_features = categorical_indices, random_state = 42)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train, y_train)

In [ ]:
# Perform SMOTETomek resampling

smotetomek = SMOTETomek(smote = smotenc, random_state = 42)
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(X_train, y_train)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [ ]:
# Train logistic regression with One-vs-Rest strategy for data balanced with smotenc

log_reg = LogisticRegression(solver='liblinear')
ovr_smotenc = OneVsRestClassifier(log_reg)
ovr_smotenc.fit(X_train_smotenc, y_train_smotenc)
ovr_smotenc_predictions = ovr_smotenc.predict(X_test)

# Calculate metrics

print(classification_report(y_test, ovr_smotenc_predictions))

              precision    recall  f1-score   support

           A       0.43      0.47      0.45       394
           B       0.38      0.23      0.29       372
           C       0.50      0.59      0.54       394
           D       0.67      0.73      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.50      0.49      1614
weighted avg       0.50      0.52      0.50      1614



In [ ]:
# Train logistic regression with One-vs-Rest strategy for data balanced with smotetomek

log_reg = LogisticRegression(solver='liblinear')
ovr_smotetomek = OneVsRestClassifier(log_reg)
ovr_smotetomek.fit(X_train_smotetomek, y_train_smotetomek)
ovr_smotetomek_predictions = ovr_smotetomek.predict(X_test)

# Calculate metrics

print(classification_report(y_test, ovr_smotetomek_predictions))

              precision    recall  f1-score   support

           A       0.43      0.46      0.45       394
           B       0.38      0.26      0.31       372
           C       0.51      0.58      0.54       394
           D       0.67      0.72      0.69       454

    accuracy                           0.52      1614
   macro avg       0.50      0.50      0.50      1614
weighted avg       0.50      0.52      0.51      1614



**Висновки**

Моделі показали однаковий результат на тестовому сеті (macro-average f1 складає відповідно 0.49 та 0.5.

Одна з вірогідних причин це те, що кількість зв'язків Томека невелика, тому здебільшого на даних відбувався ресемплінг Smotenc.